# Iterators

Iterators are everywhere in Python. They are elegantly implemented within for loops, comprehensions, generators, etc. They are simply objects that can be iterated upon. An object that will return data, one element at a time.

Technically speaking, a Python iterator object must implement two special methods, `__iter__()` and `__next__()`, collectively called the iterator protocol.

An object is called iterable if we can get an iterator from it. Most of built-in containers in Python like `list`, `tuple`, `string` etc. are iterables. The `iter()` function (which in turn calls the `__iter__()` method) returns an iterator from them.

We use the `next()` function to manually iterate through all the items of an iterator. When we reach the end and there is no more data to be returned, it will raise `StopIteration`.

In [1]:
# define a list
my_list = [4, 7, 0, 3]

# get an iterator using iter()
my_iter = iter(my_list)

## iterate through it using next() 

#prints 4
print(next(my_iter))

#prints 7
print(next(my_iter))

## next(obj) is same as obj.__next__()

#prints 0
print(my_iter.__next__())

#prints 3
print(my_iter.__next__())

## This will raise error, no items left
next(my_iter)

4
7
0
3


StopIteration: 

# How Does a `for` Loop Work?

The `for` loop can iterate over any iterable. Let's take a closer look at how the `for` loop is actually implemented in Python.

In [2]:
for element in iterable:
    # do something with element

SyntaxError: unexpected EOF while parsing (<ipython-input-2-b59d66505d29>, line 2)

is actually implemented as

In [3]:
# create an iterator object from that iterable
iter_obj = iter(iterable)
# infinite loop
while True:
    try:
        # get the next item
        element = next(iter_obj)
        # do something with element
    except StopIteration:
        # if StopIteration is raised, break from loop
        break

NameError: name 'iterable' is not defined

# Building Your Own Iterator in Python

Building an iterator from scratch is easy in Python. We just have to implement the methods `__iter__()` and `__next__()`.

The `__iter__()` method returns the iterator object itself. If required, some initialization can be performed.

The `__next__()` method must return the next item in the sequence. On reaching the end, and in subsequent calls, it must raise StopIteration.

Here is an example that will give us the next power of 2 in each iteration. The power exponent starts from zero up to a user set number.

In [ ]:
class PowerTwo:
    """Class to implement an iterator
    of powers of two"""

    def __init__(self, max = 0):
        self.max = max

    def __iter__(self):
        self.n = 0
        return self

    def __next__(self):
        if self.n <= self.max:
            result = 2 ** self.n
            self.n += 1
            return result
        else:
            raise StopIteration

Now, create an iterator and iterate through it.

In [ ]:
a = PowerTwo(4)
i = iter(a)
next(i)
next(i)
next(i)
next(i)
next(i)

We can also use a `for` loop to iterate over our iterator class.

In [ ]:
for i in PowerTwo(5):
    print(i)

# What are Generators in Python?

There is a lot of overhead in building an iterator in Python; we have to implement a class with `__iter__()` and `__next__()` methods, keep track of internal states, raise `StopIteration` when there was no values to be returned, etc. Python generators are a simple way of creating iterators. All the overheads we mentioned above are automatically handled by generators in Python. 

Here is how a generator function differs from a normal function.

1. A generator function contains one or more `yield` statements.
2. When called, it returns an object (iterator) but does not start execution immediately.
Methods like `__iter__()` and `__next__()` are implemented automatically. So we can iterate through the items using `next()`.
3. Once the function yields, the function is paused and the control is transferred to the caller.
4. While a `return` statement terminates a function entirely, a `yield` statement pauses the function saving all its states and later continues from there on successive calls.
5. Finally, when the function terminates, `StopIteration` is raised automatically on further calls.

Here is an example to illustrate all of the points stated above. We have a generator function named `my_gen()` with several yield statements.

In [1]:
# A simple generator function
def my_gen():
    n = 1
    print('This is printed first')
    # Generator function contains yield statements
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n
    
a = my_gen()
next(a)

for item in my_gen():
    print(item)

This is printed first
This is printed first
1
This is printed second
2
This is printed at last
3


# Generators are Easier to Code

Generators are easier to code than iterators as illustrated below in our powers of two example.

In [2]:
def PowerTwoGen(max = 0):
    n = 0
    while n < max:
        yield 2 ** n
        n += 1
        
for i in PowerTwoGen(5):
    print(i)

1
2
4
8
16


# Generator Expressions

Like list comprehensions, generator expressions allow you to quickly create a generator object in just a few lines of code. They’re also useful in the same cases where list comprehensions are used, with an added benefit: you can create them without building and holding the entire object in memory before iteration. In other words, you’ll have no memory penalty when you use generator expressions. Take this example of squaring some numbers:

In [3]:
nums_squared_lc = [num**2 for num in range(5)]
nums_squared_gc = (num**2 for num in range(5))

In [4]:
nums_squared_lc

[0, 1, 4, 9, 16]

In [5]:
nums_squared_gc

<generator object <genexpr> at 0x000002AEA0F822A0>

The first object used brackets to build a list, while the second created a generator expression by using parentheses. The output confirms that you’ve created a generator object and that it is distinct from a list.

# Generators Handle Infinitely Many Values

With a generator, we can avoid needing to store all generated values simultaneously, which can bring significant memory savings. In fact, it might be *impossible* to store all values generated!  A generator may yield *infinitely* many values.

Since we usually want our programs to finish, how could it be useful to generate infinitely many values?  Check out this example.  We will take a roundabout path to finding the first few prime numbers.

In [9]:
def all_positive_integers():
    """Generator for all numbers greater than zero"""
    
    i = 1
    while True:
        # Note: loop is intentionally nonterminating!
        yield i
        i += 1

Yes, a naive reading of `all_positive_integers` shows it running forever.  However, it continually invokes `yield`, which *suspends execution of the generator, until the consumer is ready for the next value*.  A consumer that only asks for finitely many values can terminate, even when the generator has the *capacity* to produce infinitely many values.  

# Pipelining Generators

With generators, we can chain together a set of very small and concise processes over a set of items, with the initial input being infinitely many!

In [10]:
def firstn(stream, num):
    """Return the first num (which must be positive) values generated by stream."""
    
    for v in stream:
        yield v
        num -= 1
        if num <= 0:
            return

Here is a simple pipeline, where we simply count up to the number of items we want to return.

In [11]:
list(firstn(all_positive_integers(), 10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Note in the above example that we are providing `list` with a generator object. Note that only one number is processed at a time, i.e., the first number is generated by `all_positive_integers()` and then goes through `firstN()` before control goes back to `all_positive_integers()`. Here are more interesting examples below that use generator expressions. Notice the `()` around the first argument of `firstn` signifying a generator object.

In [12]:
list(firstn((n for n in all_positive_integers() if n % 2 == 0), 10))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

We can also implement primality testing to get the first few primes.

In [13]:
def is_prime(n):
    return n > 1 and all(n % m != 0 for m in range(2, n))

list(firstn((n for n in all_positive_integers() if is_prime(n)), 20))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

# Profiling Generator Performance

Generators are a great way to optimize memory. While an infinite sequence generator is an extreme example of this optimization, let’s amp up the number squaring examples you just saw and inspect the size of the resulting objects. You can do this with a call to `sys.getsizeof()`:

In [14]:
import sys
nums_squared_lc = [i ** 2 for i in range(10000)]
sys.getsizeof(nums_squared_lc)

87624

In [15]:
nums_squared_gc = (i ** 2 for i in range(10000))
print(sys.getsizeof(nums_squared_gc))

120


There is one thing to keep in mind, though. If the list is smaller than the running machine’s available memory, then list comprehensions can be faster to evaluate than the equivalent generator expression. To explore this, let’s sum across the results from the two number squaring examples above. You can generate a readout with `cProfile.run()`:

In [16]:
import cProfile
cProfile.run('sum([i ** 2 for i in range(10000)])')

         5 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.004    0.004    0.004    0.004 <string>:1(<listcomp>)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [17]:
cProfile.run('sum((i ** 2 for i in range(10000)))')

         10005 function calls in 0.007 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.006    0.000    0.006    0.000 <string>:1(<genexpr>)
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        1    0.000    0.000    0.007    0.007 {built-in method builtins.exec}
        1    0.001    0.001    0.007    0.007 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Here, you can see that summing across all values in the list comprehension took less time than summing across the generator. If speed is an issue and memory isn’t, then a list comprehension is likely a better tool for the job.

# Working with Files

A common use case of generators is to work with data streams or large files, like CSV files. These text files separate data into columns by using commas. This format is a common way to share data. Now, what if you want to count the number of rows in a CSV file? The code block below shows one way of counting those rows:

In [18]:
def csv_reader(file_name):
    file = open(file_name)
    result = file.read().split()
    return result

This function opens a given file and uses `file.read()` along with `.split()` to add each line as a separate element to a list. 

In [19]:
csv_gen = csv_reader("small_file.txt")
row_count = 0

for row in csv_gen:
    row_count += 1

print(f"Row count is {row_count}")

Row count is 4


In this case, `open()` returns a generator object that you can lazily iterate through line by line. However, `file.read().split()` loads everything into memory at once, causing a `MemoryError` for large files. We should use generators.

In [20]:
def csv_reader(file_name):
    for row in open(file_name, "r"):
        yield row

If we use generator expressions we do not need `csv_reader()`.

In [21]:
csv_gen = (row for row in open("small_file.txt"))
row_count = 0

for row in csv_gen:
    row_count += 1

print(f"Row count is {row_count}")

Row count is 4


Hopefully, you are now convinced how generators make life easier for Python programmers! 

# Revisiting Binary Search Tree Traversal

Let's revisit a Binary Search Tree (BST) example from earlier in the term where we showed you iterators and generators to produce to traverse the tree.

In [22]:
class BST:
    def __init__(self, key, value, left=None, right=None):
        self.key = key
        self.value = value
        self.left = left
        self.right = right

def example():
    return BST(5, 'D',
        BST(3, 'B',
           BST(1, 'A'),
           BST(4, 'C')
        ),
        BST(11, 'F',
           BST(9, 'E'),
           BST(23, 'G')
        )
    )

Here is how we iterate over the elements of a BST using an inorder traversal.

In [23]:
class BST(BST):
    def items(self):
        """Return a list of (key, value) pairs in order by key"""
        items = []
        def recurse(node):
            if node is None: return
            recurse(node.left)
            items.append((node.key, node.value))
            recurse(node.right)
        recurse(self)
        return items

total = 0
for key, value in example():
    total += key
print("Total key value:", total)

TypeError: 'BST' object is not iterable

What happened here? We need to add an iterator method!

In [24]:
class BST(BST):
    def __iter__(self):
        return iter(self.items())

total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56


Now let's try to rewrite `__iter__` without generating a list, and instead using a generator.  

In [25]:
class BST(BST):
    def __iter__(self): # not a generator, but returns one
        """Return an iterator of (key, value) pairs in order by key"""
        def recurse(node): # generator
            if node is None: return
            recurse(node.left)
            yield (node.key, node.value)
            recurse(node.right)
        return recurse(self)
    
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 5


What happened here? Our code needs to be fixed because, when we recurse on one of the subtrees, we get a generator of the results, so we need to iterate through it and `yield` each one to send it to the next level up.

In [26]:
class BST(BST):
    def __iter__(self): # not a generator, but returns one
        """Return an iterator of (key, value) pairs in order by key"""
        def recurse(node): # generator
            if node is None: return
            for x in recurse(node.left):
                yield x
            yield (node.key, node.value)
            for x in recurse(node.right):
                yield x
        return recurse(self)
    
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56
